Установить библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указать версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html



In [1]:
!pip install pandas==1.5.3 pyreadstat==1.2.0 tqdm==4.65.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.5 MB/s eta 0:00:00


Подключить Google Disk, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него. Файлы *.sav должны быть заранее загружены на Google Disk в папку коуж. Если они загружены по другому пути, надо будет поменять переменную fpath (cм. ниже)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from collections import defaultdict

import pandas as pd
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat
from tqdm.notebook import tqdm as tqdm

In [4]:
fpath_diary_2019 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_DIARYSVODO_2019.sav' # Источник данных: https://gks.ru/ .
df_diary_2019, meta_diary_2019 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_diary_2019, num_processes=16, encoding='cp1251') 
meta_diary_2019.column_names_to_labels

{'H00_02': 'Код субъекта РФ',
 'H00_04': 'Код типа населенного пункта',
 'H00_06': 'Номер домохозяйства в пределах территории',
 'H00_07': 'H00_07',
 'H01_00': 'Индивидуальный код члена домохозяйства',
 'H01_01': 'Пол респондента',
 'H01_02': 'Число исполнившихся лет',
 'D01': 'День недели, в течение которого заполняется дневник',
 'K_110': 'Работа в корпорациях, органах государственного управления и некоммерческих организациях',
 'K_121': 'Выращивание сельскохозяйственных культур на домашних предприятиях для реализации на рынке',
 'K_122': 'Разведение животных на домашних предприятиях для реализации на рынке',
 'K_123': 'Лесоводство и лесозаготовки на домашних предприятиях для реализации на рынке',
 'K_124': 'Ловля рыбы на домашних предприятиях для реализации на рынке',
 'K_125': 'Аквакультура на домашних предприятиях для реализации на рынке',
 'K_126': 'Подземная и открытая разработка полезных ископаемых на домашних предприятиях для реализации на рынке',
 'K_127': 'Производство и пер

In [5]:
selection = [
    'H00_02', # 'Код субъекта РФ',
    'H01_00', # 'Индивидуальный код члена домохозяйства',
    'D01', # 'День недели, в течение которого заполняется дневник',
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'H00_06', # 'Номер домохозяйства в пределах территории',


    'K_181', # 'Передвижения, связанные с работой',
    'K_640', #'Время, затраченное на передвижения в связи с обучением',
    'K_750', #'Время, затраченное на передвижения в связи с общением и взаимодействием с людьми, участием в общественной жизни и отправлением религиозног',
    'K_860', # 'Время, затраченное на передвижения в связи с участием в культурной жизни, досугом, использованием средств массовой информации и занятиями ?',
    'K_950', #'Время, затраченное на передвижения в связи с уходом за собой',
    'K_997', #'Передвижение, связанное с многоцелевой деятельностью и сменой местности',
    'K_250', #'Передвижения, перемещения, перевозка или сопровождение товаров или людей в связи с производством товаров для собственного использования',
    'K_380', #'Поездки, передвижения, перевозка или сопровождение товаров или людей в связи с оказанием неоплачиваемых бытовых услуг членам домохозяйств',
    'K_441', #'Передвижения, связанные с оказанием услуг по уходу за членами домохозяйства и семьи',
    'K_540', #'Время, затраченное на передвижения в связи с неоплачиваемой трудовой деятельностью волонтеров, стажеров и другими видами неоплачиваемой т',
    ] 

In [6]:
def select_variables(df, selection, meta):
    selected = df[selection]
    selection_names = [meta.column_names_to_labels[label].strip() for label in selection]
    selected.columns = selection_names
    return selected

Отличие этих данных от данных КОУЖ: для одного человека может быть более одной строки, так как один и тот же человек заполняет свой дневник в разные дни недели. Поэтому:

1. Группируем по кодам субъекта РФ
2. Группируем по домохозяйствам
3. Группируем по членам домохозяйства
4. Суммируем значения для всех дней недели по интересующим нас переменным
5. Теперь, когда для каждого человека одна строка, можно считать описательные статистики


In [7]:
def select_region(df, code):
    mask = df['Код субъекта РФ'] == code
    region = df[mask]
    print(f"Количество наблюдений по региону {code}: {region.shape[0]}")
    return region

In [8]:
whole_russia_2019 = select_variables(df_diary_2019, selection, meta_diary_2019)
whole_russia_2019 = whole_russia_2019[whole_russia_2019['Место проживания']==1]
my_region_2019 = select_region(whole_russia_2019, '98') # заменить номер региона на свой
my_region_2019.head()

Количество наблюдений по региону 98: 755


,Код субъекта РФ,Индивидуальный код члена домохозяйства,"День недели, в течение которого заполняется дневник",Место проживания,Размер населенного пункта,Номер домохозяйства в пределах территории,"Передвижения, связанные с работой","Время, затраченное на передвижения в связи с обучением","Время, затраченное на передвижения в связи с общением и взаимодействием с людьми, участием в общественной жизни и отправлением религиозног","Время, затраченное на передвижения в связи с участием в культурной жизни, досугом, использованием средств массовой информации и занятиями ?","Время, затраченное на передвижения в связи с уходом за собой","Передвижение, связанное с многоцелевой деятельностью и сменой местности","Передвижения, перемещения, перевозка или сопровождение товаров или людей в связи с производством товаров для собственного использования","Поездки, передвижения, перевозка или сопровождение товаров или людей в связи с оказанием неоплачиваемых бытовых услуг членам домохозяйств","Передвижения, связанные с оказанием услуг по уходу за членами домохозяйства и семьи","Время, затраченное на передвижения в связи с неоплачиваемой трудовой деятельностью волонтеров, стажеров и другими видами неоплачиваемой т"
150693,98,1.0,2.0,1.0,1.0,44451.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150694,98,1.0,7.0,1.0,1.0,44451.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
150695,98,1.0,1.0,1.0,1.0,44452.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
150696,98,1.0,7.0,1.0,1.0,44452.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
150697,98,1.0,2.0,1.0,1.0,44453.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0


In [9]:
my_region_2019_sums = defaultdict(list)
for household in my_region_2019['Номер домохозяйства в пределах территории'].unique():
    current_household = my_region_2019[my_region_2019['Номер домохозяйства в пределах территории'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        my_region_2019_sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        for column in current_member.columns[6:]:
            my_region_2019_sums[column].append(current_member[column].sum())
my_region_2019_sums = pd.DataFrame(my_region_2019_sums)
# Описательные статистики для городов разного размера данного региона
for size in my_region_2019_sums['Размер населенного пункта'].unique():
    print(meta_diary_2019.value_labels[meta_diary_2019.variable_to_label['R_1_1']][size])
    my_region_2019_sums_by_size = my_region_2019_sums[my_region_2019_sums['Размер населенного пункта']==size]
    for variable in my_region_2019_sums_by_size.columns[1:]:
        print(variable)
        print(my_region_2019_sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

городской, менее 50,0 тыс.чел.
Передвижения, связанные с работой
count    235.000000
mean       0.553191
std        4.542339
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       50.000000
Name: Передвижения, связанные с работой, dtype: float64
Время, затраченное на передвижения в связи с обучением
count    235.000000
mean       6.255319
std       24.486581
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      270.000000
Name: Время, затраченное на передвижения в связи с обучением, dtype: float64
Время, затраченное на передвижения в связи с общением и взаимодействием с людьми, участием в общественной жизни и отправлением религиозног
count    235.000000
mean      15.319149
std       25.658096
min        0.000000
25%        0.000000
50%        0.000000
75%       30.000000
max      130.000000
Name: Время, затраченное на передвижения в связи с общением и взаимодействием с людьми, участием в общественной жизни и отправле

## Всё то же самое, но для 2014 года

In [10]:
fpath_2014 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_Diary_public_2014.sav'
df_2014, meta_2014 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2014, num_processes=16, encoding='cp1251') 
meta_2014.column_names_to_labels

{'HH_number': 'Номер домохозяйства (сквозной)',
 'D00_08': 'Индивидуальный код члена домохозяйства',
 'BD': 'Будний день',
 'RD': 'Рабочий день',
 'PVD': 'Предвыходной день',
 'VD': 'Выходной день',
 'K_10': 'Рабочее время на основной работе в формальном секторе',
 'K_20': 'Рабочее время на дополнительной работе в формальном секторе',
 'K_30': 'Рабочее время в качестве ученика, стажера, подмастерья в формальном секторе',
 'K_40': 'Короткие перерывы в работе в формальном секторе',
 'K_50': 'Обучение,  профподготовка/переподготовка  в связи с работой в «формальном секторе»',
 'K_80': 'Деятельность, связанная с поиском работы (для желающих работать в формальном секторе)',
 'K_90': 'Организация работы предприятия, деятельность по созданию предприятия в формальном секторе',
 'K_100': 'Передвижения, связанные с работой в «формальном секторе»',
 'K_110': 'Работав «формальномсекторе», неотнесеннаякдругимкатегориям',
 'K_121': 'Освоение земельного участка (основная работа)',
 'K_122': 'Освоение

In [11]:
selection_2014 = [ # набор переменных в 2014 году отличается от 2019
    'D00_08', # 'Индивидуальный код члена домохозяйства',
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'HH_number', #'Номер домохозяйства (сквозной)'

     'K_100', # 'Передвижения, связанные с работой в «формальном секторе»',
     'K_500', # 'Передвижения, связанные с производством первичной продукции (основная работа)',
     'K_511', # 'Передвижения, связанные с производством первичной продукции (дополнительная работа)',
     'K_512', # 'Передвижения, связанные с производством первичной продукции (для себя)',
     'K_854', # 'Передвижения, связанные с производством продукции, не являющейся первичной(основная работа)',
     'K_861', # 'Передвижения, связанные с производством продукции, не являющейся первичной (дополнительная работа)',
     'K_870', # 'Передвижения, связанные с производством продукции, не являющейся первичной (для себя)',
     'K_964', # 'Передвижения, связанные с деятельностью в области строительства (основная работа)',
     'K_971', # 'Передвижения, связанные с деятельностью в области строительства (дополнительная работа)',
     'K_972', # 'Передвижения, связанные с деятельностью в области строительства (для себя)',
     'K_1222', # 'Передвижения, связанные с предоставлением услуг в целях получения дохода (основная работа)',
     'K_1231', # 'Передвижения, связанные с предоставлением услуг в целях получения дохода (дополнительная работа)',
     'K_1530', # 'Передвижения, связанные с оказанием неоплачиваемых бытовых услуг',
     'K_1610', # 'Передвижения, связанные с оказанием неоплачиваемых услуг по уходу за членами домохозяйства',
     'K_1690', # 'Передвижения, связанные с оказанием услуг обществу и помощи другим домохозяйствам',
     'K_1800', # 'Передвижения, связанные с обучением',
     'K_1910', # 'Передвижения, связанные с общением и участием в общественной жизни',
     'K_1990', # 'Передвижения, связанные с посещением культурных, развлекательных и спортивных мероприятий/мест отдыха',
     'K_2040', # 'Передвижения, связанные с хобби, играми и другими видами досуга',
     'K_2090', # 'Передвижения,связанные с занятиями спортом и туризмом',
     'K_2200', # 'Передвижения, связанные с обращением к средствам массовой информации',
     'K_2420', # 'Передвижения, связанные с личной гигиеной,  уходом за собой, деятельностью религиозного характера, обрядами и др.  культовыми мероприятиями',
    ] 
whole_russia_2014 = select_variables(df_2014, selection_2014, meta_2014)
# в данных за 2014 нет кода субъекта РФ
sums = defaultdict(list)

for household in tqdm(whole_russia_2014['Номер домохозяйства (сквозной)'].unique()): 
    current_household = whole_russia_2014[whole_russia_2014['Номер домохозяйства (сквозной)'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        for column in current_member.columns[4:]:
            sums[column].append(current_member[column].sum())
sums = pd.DataFrame(sums)
for size in sums['Размер населенного пункта'].unique():
    print(meta_2014.value_labels[meta_2014.variable_to_label['R_1_1']][size])
    sums_by_size = sums[sums['Размер населенного пункта']==size]
    for variable in sums_by_size.columns[1:]:
        print(variable)
        print(sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

  0%|          | 0/9996 [00:00<?, ?it/s]

менее 50,0 тыс. человек
Передвижения, связанные с работой в «формальном секторе»
count    2829.000000
mean       30.031813
std        45.656058
min         0.000000
25%         0.000000
50%         0.000000
75%        50.000000
max       760.000000
Name: Передвижения, связанные с работой в «формальном секторе», dtype: float64
Передвижения, связанные с производством первичной продукции (основная работа)
count    2829.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Передвижения, связанные с производством первичной продукции (основная работа), dtype: float64
Передвижения, связанные с производством первичной продукции (дополнительная работа)
count    2829.000000
mean        0.081301
std         2.199500
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       100.000000
Name: Передвижения, связанные с производством первичной продукции (дополнительная работа), dtype: float64
Передвиж